# imports, config etc.

In [2]:
# !pip install dataclasses-json
# !pip freeze | grep -i dataclasses-json= >> requirements.txt
# !cat requirements.txt

In [1]:
import asyncio
import os
from pprint import pp
from typing import Any, AsyncIterator, Callable, Coroutine, Literal, Self

from IPython.display import Markdown, HTML, display
import ipywidgets as widgets

from dotenv import load_dotenv

load_dotenv()

True

# WIP

In [2]:
from summony.agents import OpenAIAgent
        

ag = OpenAIAgent('gpt-4o')

########## SYNC
r = ag.ask("List maxwell's equations. Use markdown, and latex with only $ (dollar-sign) delimiters (eg. $x_i$).")
print(r)
display(HTML('<hr>'))
display(Markdown(r.replace('\n', '\n\n')))

pp((ag.messages, ag.raw_responses))

r = ag.ask("Which of them do you thin is the most consequential and why?")
print(r)
display(HTML('<hr>'))
display(Markdown(r.replace('\n', '\n\n')))

pp((ag.messages, ag.raw_responses))

# ########## ASYNC STREAM
# r = ag.ask_async_stream("List maxwell's equations. Use markdown, and latex with only $ (dollar-sign) delimiters (eg. $x_i$). Please NEVER use `\\[` delimiters for math!")
# async for c in r:
#     print(c, end='')
# display(HTML('<hr>'))
# display(Markdown(ag.messages[-1].content.replace('\n', '\n\n')))

# pp((ag.messages, ag.raw_responses.keys()))

# r = ag.ask_async_stream("Which of them do you thin is the most consequential and why?")
# async for c in r:
#     print(c, end='')
# display(HTML('<hr>'))
# display(Markdown(ag.messages[-1].content.replace('\n', '\n\n')))

# pp((ag.messages, ag.raw_responses.keys()))

Certainly! Here are Maxwell's equations written in Markdown format with LaTeX using dollar-sign delimiters:

1. **Gauss's Law for Electricity:**
   $$ \nabla \cdot \mathbf{E} = \frac{\rho}{\epsilon_0} $$

2. **Gauss's Law for Magnetism:**
   $$ \nabla \cdot \mathbf{B} = 0 $$

3. **Faraday's Law of Induction:**
   $$ \nabla \times \mathbf{E} = -\frac{\partial \mathbf{B}}{\partial t} $$

4. **Ampère's Law (with Maxwell's correction):**
   $$ \nabla \times \mathbf{B} = \mu_0 \mathbf{J} + \mu_0 \epsilon_0 \frac{\partial \mathbf{E}}{\partial t} $$

These are the fundamental equations governing electromagnetism.


Certainly! Here are Maxwell's equations written in Markdown format with LaTeX using dollar-sign delimiters:



1. **Gauss's Law for Electricity:**

   $$ \nabla \cdot \mathbf{E} = \frac{\rho}{\epsilon_0} $$



2. **Gauss's Law for Magnetism:**

   $$ \nabla \cdot \mathbf{B} = 0 $$



3. **Faraday's Law of Induction:**

   $$ \nabla \times \mathbf{E} = -\frac{\partial \mathbf{B}}{\partial t} $$



4. **Ampère's Law (with Maxwell's correction):**

   $$ \nabla \times \mathbf{B} = \mu_0 \mathbf{J} + \mu_0 \epsilon_0 \frac{\partial \mathbf{E}}{\partial t} $$



These are the fundamental equations governing electromagnetism.

([Message(role='user',
          content="List maxwell's equations. Use markdown, and latex with only "
                  '$ (dollar-sign) delimiters (eg. $x_i$).'),
  Message(role='assistant',
          content="Certainly! Here are Maxwell's equations written in Markdown "
                  'format with LaTeX using dollar-sign delimiters:\n'
                  '\n'
                  "1. **Gauss's Law for Electricity:**\n"
                  '   $$ \\nabla \\cdot \\mathbf{E} = '
                  '\\frac{\\rho}{\\epsilon_0} $$\n'
                  '\n'
                  "2. **Gauss's Law for Magnetism:**\n"
                  '   $$ \\nabla \\cdot \\mathbf{B} = 0 $$\n'
                  '\n'
                  "3. **Faraday's Law of Induction:**\n"
                  '   $$ \\nabla \\times \\mathbf{E} = -\\frac{\\partial '
                  '\\mathbf{B}}{\\partial t} $$\n'
                  '\n'
                  "4. **Ampère's Law (with Maxwell's correction):**\n"
                  '   $$ 

The significance of each of Maxwell's equations can vary depending on the context, but many physicists and engineers might argue that **Faraday's Law of Induction** stands out as particularly consequential for several reasons:



1. **Electromagnetic Induction:**

   $$ \nabla \times \mathbf{E} = -\frac{\partial \mathbf{B}}{\partial t} $$

   Faraday's Law describes how a time-varying magnetic field induces an electric field. This principle is the foundation of many technologies we use today, such as electric generators and transformers. The ability to convert mechanical energy into electrical energy (and vice versa) relies on Faraday's Law.



2. **Technological Applications:**

   - **Electric Generators:** Utilize the principle of electromagnetic induction to produce electrical energy from kinetic energy.

   - **Transformers:** Operate on the principle of induction to step up or step down voltage levels in power transmission.

   - **Inductive Charging:** Used in wireless charging technologies for electronic devices.



3. **Fundamental Insight:**

   Faraday's Law provides a profound insight into the interdependence of electric and magnetic fields, which is a cornerstone of classical electromagnetism and a precursor to the concept of electromagnetic waves.



4. **Inspiration for Electromagnetic Waves:**

   Along with Ampère's Law with Maxwell's correction, Faraday's Law leads to the understanding that changing electric and magnetic fields propagate as waves (electromagnetic waves). This was crucial in the development of the theory of light and radio waves.



While each of Maxwell's equations is indispensable in its own right and describes a different aspect of electromagnetism, Faraday's Law of Induction is frequently highlighted due to its wide-ranging practical applications and its importance in the generation and distribution of electrical energy.

([Message(role='user',
          content="List maxwell's equations. Use markdown, and latex with only "
                  '$ (dollar-sign) delimiters (eg. $x_i$).'),
  Message(role='assistant',
          content="Certainly! Here are Maxwell's equations written in Markdown "
                  'format with LaTeX using dollar-sign delimiters:\n'
                  '\n'
                  "1. **Gauss's Law for Electricity:**\n"
                  '   $$ \\nabla \\cdot \\mathbf{E} = '
                  '\\frac{\\rho}{\\epsilon_0} $$\n'
                  '\n'
                  "2. **Gauss's Law for Magnetism:**\n"
                  '   $$ \\nabla \\cdot \\mathbf{B} = 0 $$\n'
                  '\n'
                  "3. **Faraday's Law of Induction:**\n"
                  '   $$ \\nabla \\times \\mathbf{E} = -\\frac{\\partial '
                  '\\mathbf{B}}{\\partial t} $$\n'
                  '\n'
                  "4. **Ampère's Law (with Maxwell's correction):**\n"
                  '   $$ 

In [3]:
from summony.agents import get_default_agent_for_model
        
ag = get_default_agent_for_model('claude-3-5-sonnet')

# ########## SYNC
# r = ag.ask("List maxwell's equations. Use markdown, and latex with only $ (dollar-sign) delimiters (eg. $x_i$).")
# print(r)
# display(HTML('<hr>'))
# display(Markdown(r.replace('\n', '\n\n')))

# pp((ag.messages, ag.raw_responses))

# r = ag.ask("Which of them do you thin is the most consequential and why?")
# print(r)
# display(HTML('<hr>'))
# display(Markdown(r.replace('\n', '\n\n')))

# pp((ag.messages, ag.raw_responses))

# ########## ASYNC STREAM
# r = ag.ask_async_stream("List maxwell's equations. Use markdown, and latex with only $ (dollar-sign) delimiters (eg. $x_i$). Please NEVER use `\\[` delimiters for math!")
# async for c in r:
#     print(c, end='')
# display(HTML('<hr>'))
# display(Markdown(ag.messages[-1].content.replace('\n', '\n\n')))

# pp((ag.messages, ag.raw_responses.keys()))

# r = ag.ask_async_stream("Which of them do you thin is the most consequential and why?")
# async for c in r:
#     print(c, end='')
# display(HTML('<hr>'))
# display(Markdown(ag.messages[-1].content.replace('\n', '\n\n')))

# pp((ag.messages, ag.raw_responses.keys()))

In [4]:
from summony.uis.nbui import NBUI

c = NBUI(models=["gpt-4", "claude-3-opus-20240229"])
# c = NBUI(models=["gpt-4"])
await c("List maxwell's equations. Use markdown and latex with $ delimiters.")

Accordion(children=(HTML(value=''),), selected_index=0, titles=('raw replies',))

1. Gauss's Law of Electricity: 



$$\nabla \cdot \vec{E} = \frac{\rho}{\varepsilon_0}$$



2. Gauss's Law of Magnetism:



$$\nabla \cdot \vec{B} = 0$$



3. Faraday's Law of Induction:



$$\nabla \times \vec{E} = -\frac{\partial \vec{B}}{\partial t}$$



4. Ampère's Circuital Law (with Maxwell's addition):



$$\nabla \times \vec{B} = \mu_0 \vec{J} + \mu_0 \varepsilon_0 \frac{\partial \vec{E}}{\partial t}$$



- $\vec{E}$ is the electric field vector

- $\vec{B}$ is the magnetic field vector

- $\vec{J}$ is the current density vector

- $\rho$ is the electric charge density

- $\mu_0$ is the permeability in vacuum

- $\varepsilon_0$ is the permittivity in vacuum

- $\nabla \cdot$ and $\nabla \times$ are divergence and curl operators respectively

- Time $t$

Here are Maxwell's equations using markdown and LaTeX with $ delimiters:



1. Gauss's law for electric fields:

   

   $\nabla \cdot \mathbf{E} = \frac{\rho}{\varepsilon_0}$



2. Gauss's law for magnetic fields:

   

   $\nabla \cdot \mathbf{B} = 0$



3. Faraday's law of induction:

   

   $\nabla \times \mathbf{E} = -\frac{\partial \mathbf{B}}{\partial t}$



4. Ampère's circuital law (with Maxwell's correction):

   

   $\nabla \times \mathbf{B} = \mu_0 \left(\mathbf{J} + \varepsilon_0 \frac{\partial \mathbf{E}}{\partial t}\right)$



Where:

- $\mathbf{E}$ is the electric field vector

- $\mathbf{B}$ is the magnetic field vector

- $\rho$ is the electric charge density

- $\mathbf{J}$ is the electric current density

- $\varepsilon_0$ is the permittivity of free space

- $\mu_0$ is the permeability of free space

- $\nabla \cdot$ is the divergence operator

- $\nabla \times$ is the curl operator

- $\frac{\partial}{\partial t}$ is the partial derivative with respect to time

# EOF